# Design

This is an decoder model which will take in a roundness value and output a pseudoword that corresponds to the roundness value

In [1]:
from utils.model import *
from utils.model_utils import *
import pandas as pd
import torch

pd.set_option('display.max_columns', None)
device = "cuda" if torch.cuda.is_available() else "cpu"
state = 42

c:\Users\qkm20\anaconda3\envs\pseudoword\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

In [2]:
# Import dataset

data = pd.read_csv('datasets/normalized.csv')
data

,Stimuli,ExperimentalRoundScore
0,bebi,0.815217
1,bibe,0.913043
2,bobou,0.815217
3,boubo,1.000000
4,chechi,0.184783
...,...,...
119,outou,0.347826
120,uku,0.239130
121,ulu,0.913043
122,umu,0.913043


In [3]:
import random

def augment_data(data, num_augmentations=2):
    augmented_data = data.copy()
    for _ in range(num_augmentations):
        new_rows = []
        for _, row in data.iterrows():
            new_stimuli = list(row['Stimuli'])
            # Randomly change one character in the pseudoword
            idx = random.randint(0, len(new_stimuli) - 1)
            new_stimuli[idx] = chr(random.randint(97, 122))  # Replace with a random lowercase letter
            new_stimuli = ''.join(new_stimuli)
            new_rows.append({'Stimuli': new_stimuli, 'ExperimentalRoundScore': row['ExperimentalRoundScore']})
        augmented_data = augmented_data._append(new_rows, ignore_index=True)
    return augmented_data

augmented_data = augment_data(data)
augmented_data = augment_data(augmented_data)
data = augmented_data
data

,Stimuli,ExperimentalRoundScore
0,bebi,0.815217
1,bibe,0.913043
2,bobou,0.815217
3,boubo,1.000000
4,chechi,0.184783
...,...,...
1111,futou,0.347826
1112,upf,0.239130
1113,piu,0.913043
1114,qhu,0.913043


In [4]:
# Split data into train, val and test sets

trn = data.sample(frac=0.8, random_state=state)
val = data.drop(trn.index).sample(frac=0.5, random_state=state)
tst = data.drop(trn.index).drop(val.index)
trn.reset_index(inplace=True, drop=True)
val.reset_index(inplace=True, drop=True)
tst.reset_index(inplace=True, drop=True)

# Model

In [5]:
model = RoundnessToTextModel(
    freeze_byt5=False
)

In [6]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [7]:
train(
    model=model,
    optimizer=optimizer,
    trn_roundness=trn["ExperimentalRoundScore"],
    val_roundness=val["ExperimentalRoundScore"],
    tst_roundness=tst["ExperimentalRoundScore"],
    trn_texts=trn["Stimuli"],
    val_texts=val["Stimuli"],
    tst_texts=tst["Stimuli"],
    batch_size=20,
    epochs=10,
    patience=10,
    scheduler=scheduler,
)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch   1/ 10, Train Loss: 6.8373, Validation Loss: 2.7294


c:\Users\qkm20\anaconda3\envs\pseudoword\Lib\site-packages\torch\optim\lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch   2/ 10, Train Loss: 2.1684, Validation Loss: 2.2984
Epoch   3/ 10, Train Loss: 1.9303, Validation Loss: 2.2365
Epoch   4/ 10, Train Loss: 1.7983, Validation Loss: 2.1479
Epoch   5/ 10, Train Loss: 1.6814, Validation Loss: 2.0955
Epoch   6/ 10, Train Loss: 1.5875, Validation Loss: 2.0716
Epoch   7/ 10, Train Loss: 1.4801, Validation Loss: 2.0988
Epoch   8/ 10, Train Loss: 1.4053, Validation Loss: 2.1708
Epoch   9/ 10, Train Loss: 1.3563, Validation Loss: 2.2983
Epoch  10/ 10, Train Loss: 1.3228, Validation Loss: 2.3436
Test Loss: 2.3091


In [8]:
save_model(model, filename="model_v01.pth")

# Testing

In [9]:
model = load_model(filename="model_v01.pth")

f:\OneDrive - Nanyang Technological University\FYP\Pseudoword-Generator\utils\model.py:118: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(pa

In [10]:
random_sample = tst.sample(n=5, random_state=42)

for _, row in random_sample.iterrows():
    print(f"Expected Word: {row['Stimuli']}")
    print(f"Predicted word: {inference(model, row["ExperimentalRoundScore"])}")
    print(f"Round Score: {row['ExperimentalRoundScore']}")
    print()

Expected Word: tjukeu
first few logits: tensor([ 0.8248, -0.3533,  0.5461, -0.7811,  0.6354], device='cuda:0')
tensor([[  0, 111, 114, 120, 112, 114,   1]], device='cuda:0')
Predicted word: loumo
Round Score: 0.2391304347826087

Expected Word: cjiche
first few logits: tensor([ 0.8060, -0.3531,  0.5628, -0.7819,  0.6500], device='cuda:0')
tensor([[  0, 111, 114, 120, 112, 114,   1]], device='cuda:0')
Predicted word: loumo
Round Score: 0.5434782608695652

Expected Word: neni
first few logits: tensor([ 0.8060, -0.3531,  0.5628, -0.7819,  0.6500], device='cuda:0')
tensor([[  0, 111, 114, 120, 112, 114,   1]], device='cuda:0')
Predicted word: loumo
Round Score: 0.5434782608695652

Expected Word: bosoh
first few logits: tensor([ 0.7892, -0.3528,  0.5777, -0.7826,  0.6630], device='cuda:0')
tensor([[  0, 111, 114, 120, 112, 114,   1]], device='cuda:0')
Predicted word: loumo
Round Score: 0.8152173913043478

Expected Word: logmou
first few logits: tensor([ 0.7778, -0.3527,  0.5878, -0.7830,  0.